In [1]:
import pandas as pd
import numpy as np

In [2]:
url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"
zone = pd.read_csv(url)

In [3]:
url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"
df = pd.read_csv(url,low_memory=False)

In [5]:
df.lpep_pickup_datetime=pd.to_datetime(df.lpep_pickup_datetime)
df.lpep_dropoff_datetime=pd.to_datetime(df.lpep_dropoff_datetime)

In [6]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1.0,112,196,1.0,5.88,18.00,0.50,0.5,0.00,0.00,NaN,0.3,19.30,2.0,1.0,0.0
1,1.0,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1.0,43,263,1.0,0.80,5.00,3.25,0.5,0.00,0.00,NaN,0.3,9.05,2.0,1.0,0.0
2,1.0,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1.0,255,228,2.0,7.50,21.50,0.50,0.5,0.00,0.00,NaN,0.3,22.80,2.0,1.0,0.0
3,1.0,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1.0,181,181,1.0,0.90,5.50,0.50,0.5,0.00,0.00,NaN,0.3,6.80,2.0,1.0,0.0
4,2.0,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1.0,97,188,1.0,2.52,10.00,0.50,0.5,2.26,0.00,NaN,0.3,13.56,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476381,NaN,2019-10-31 23:30:00,2019-11-01 00:00:00,NaN,NaN,65,102,NaN,7.04,29.57,2.75,0.5,0.00,0.00,NaN,0.0,32.82,NaN,NaN,NaN
476382,NaN,2019-10-31 23:03:00,2019-10-31 23:24:00,NaN,NaN,129,136,NaN,0.00,39.83,2.75,0.5,0.00,6.12,NaN,0.0,49.20,NaN,NaN,NaN
476383,NaN,2019-10-31 23:02:00,2019-10-31 23:23:00,NaN,NaN,61,222,NaN,3.90,23.11,2.75,0.5,0.00,0.00,NaN,0.0,26.36,NaN,NaN,NaN
476384,NaN,2019-10-31 23:42:00,2019-10-31 23:56:00,NaN,NaN,76,39,NaN,3.08,15.23,2.75,0.5,0.00,0.00,NaN,0.0,18.48,NaN,NaN,NaN


In [7]:
df['date_drop']=df['lpep_dropoff_datetime'].dt.date
start_date = pd.to_datetime('2019-10-01').date()
end_date = pd.to_datetime('2019-11-01').date()

In [8]:
filtered_df = df[(df['date_drop'] >= start_date) & (df['date_drop'] < end_date)]

In [9]:
def far_type(a):
    if a <= 1:
        return "<1"
    elif a <= 3:
        return "1-3"
    elif a <= 7:
        return "3-7"
    elif a <= 10:
        return "7-10"
    else:
        return "Over 10"
    
filtered_df['trip_distance_type'] = filtered_df['trip_distance'].apply(far_type)

C:\Users\PurplePolarBear\AppData\Local\Temp\ipykernel_6776\585925020.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['trip_distance_type'] = filtered_df['trip_distance'].apply(far_type)


In [10]:
filtered_df.groupby('trip_distance_type')['trip_distance_type'].count()

trip_distance_type
1-3        198924
3-7        109603
7-10        27678
<1         104802
Over 10     35189
Name: trip_distance_type, dtype: int64

In [11]:
filtered_df[filtered_df['trip_distance']==(filtered_df['trip_distance'].max())]['lpep_pickup_datetime'].dt.date

141174    2019-10-11
Name: lpep_pickup_datetime, dtype: object

In [12]:
df1=filtered_df[(filtered_df['lpep_pickup_datetime'].dt.date)==(pd.to_datetime('2019-10-18').date())]
df2 = df1.merge(zone, how='left', left_on='PULocationID', right_on='LocationID')
df3=df2.groupby(['Zone','service_zone'])['total_amount'].sum().sort_values(ascending=False).head(10).reset_index()
df3[df3['total_amount']>13000]

,Zone,service_zone,total_amount
0,East Harlem North,Boro Zone,18686.68
1,East Harlem South,Boro Zone,16797.26
2,Morningside Heights,Boro Zone,13029.79


In [13]:
filtered_df_zone = filtered_df.merge(zone, how='left', left_on='PULocationID', right_on='LocationID')
filtered_df_zone_2 = filtered_df_zone.merge(zone, how='left', left_on='DOLocationID', right_on='LocationID')

In [14]:
df5=filtered_df_zone_2[filtered_df_zone_2['Zone_x']=='East Harlem North']
max_tip=df5['tip_amount'].max()
df5[df5['tip_amount']==max_tip]['Zone_y']

308875    JFK Airport
Name: Zone_y, dtype: object